# PyTorch to ONNX Conversion

This notebook demonstrates how to convert a PyTorch model to ONNX format.

In [7]:
import torch
import torch.nn as nn
import torch.onnx

In [8]:
# Define a simple neural network model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = SimpleNet()

In [9]:
# Create a dummy input tensor (batch_size=1, input_size=10)
dummy_input = torch.randn(1, 10)

# Set the model to evaluation mode
model.eval()

# Export the model to ONNX format
torch.onnx.export(
    model,                          # model being run
    dummy_input,                    # model input (or a tuple for multiple inputs)
    "model.onnx",                   # where to save the model (can be a file or file-like object)
    export_params=True,             # store the trained parameter weights inside the model file
    opset_version=11,               # the ONNX version to export the model to
    do_constant_folding=True,       # whether to execute constant folding for optimization
    input_names=['input'],          # the model's input names
    output_names=['output'],        # the model's output names
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # variable length axes
)

print("Model exported to model.onnx")

C:\Users\emman\AppData\Local\Temp\ipykernel_32888\1332736854.py:8: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0212 20:47:56.094000 32888 Lib\site-packages\torch\onnx\_internal\exporter\_compat.py:125] Setting ONNX exporter to use operator set version 18 because the requested opset_version 11 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
C:\Python313\Lib\copyreg.py:99: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls

[torch.onnx] Obtain model graph for `SimpleNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `SimpleNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Model exported to model.onnx


In [10]:
import onnxruntime as ort
import numpy as np

# Load the ONNX model
ort_session = ort.InferenceSession("model.onnx")

# Prepare input data
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input.numpy()}

# Run inference
ort_outs = ort_session.run(None, ort_inputs)

# Compare with PyTorch output
with torch.no_grad():
    torch_out = model(dummy_input)

print("PyTorch output:", torch_out.numpy())
print("ONNX output:", ort_outs[0])
print("Outputs match:", np.allclose(torch_out.numpy(), ort_outs[0], atol=1e-5))

PyTorch output: [[-0.0576991]]
ONNX output: [[-0.0576991]]
Outputs match: True
